**Research question:**  
What asset prices move in opposition to oil prices and what is the relationship between them? 
1. Are the prices correlated? (covar)
2. Are the variances correlated (var)
3. What is the size of the correlation (beta)
4. What would be the likely outcome in the future? (MC)

The basic comparison will involve variance, covariance, and beta. Chart prices over time and overlay with oil to see if they move in inverse relation.  

Look at 3 baskets of assets that represent some sectors that I hypothesize will move countercyclicly (plastics, energy generation, airlines). 

Variance measures volatility relative to mean. If you take the var of oil, sp500, targetbasket and compare over several time periods to see if targetbasket volitility changes inline with oil. A form of countercyclicality would be if targetbasket gets less volatile while oil gets more. 

Covariance measures how correlated two assets are. This one just do oil and targetbasket. Again do over several time periods to see if covariance changes. A positive number means they move in the same direction together. Negative means they're inverse. Chart covariancgetbaskete over time.

Beta measures the scale of the change between two assets. oil as baseline asset and targetbasket. A beta of 1.0 means they're 1:1, >1 targetbasket will move more than baseline, <1 targetbasket will move less than baseline.

Could do a montecarlo using a 'portfolio' of oil and a targetbasket. If the relationship was inverse and proportional, the cumulative return would be ~0. Use beta to help find a weighting. 

In [ ]:
# Imports
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import json

In [ ]:
# Load .env environment variables, API key, dataset reference names
load_dotenv()

quandl_api_key = os.getenv("QUANDL_API_KEY")

stock_dataset = "WIKI"
oil_dataset = "EIA"

In [ ]:
# returns dataframe with the response data for the given list of tickers
def getBasket(dataset,tickers,column=1):
    df_dict = {}
    master_df = pd.DataFrame()
    for ticker in tickers: 
        target_url = f"https://www.quandl.com/api/v3/datasets/{dataset}/{ticker}.json?&column_index={column}&api_key={quandl_api_key}"
        ticker_response = requests.get(target_url).json()
        data = ticker_response['dataset']['data']
        columns = ['date',ticker]
        res_df = pd.DataFrame(data, columns=columns)
        df_dict[ticker] = res_df
    
    # Concat dfs
    for i in range(len(tickers)):
        if i == 0:
            master_df = df_dict[tickers[i]] 
        else:
            master_df = pd.concat([master_df, df_dict[tickers[i]]], axis="columns", join="inner")
            
    return master_df.T.drop_duplicates().T

In [ ]:
# takes a df and converts date to datetime and everything else to a float
def cleanTypes(df):
    for tic in df.columns:
        if tic == 'date':
            df['date'] = pd.to_datetime(df['date'].str.strip(), format='%Y/%m/%d')
        else:
            df[tic] = df[tic].astype('float')


In [ ]:
#Get oil data
oil_df = getBasket(oil_dataset, ["PET_RWTC_D"])
oil_df

In [ ]:
oil_df.isnull().sum()

In [ ]:
oil_df.duplicated().sum()

In [ ]:
oil_df.dtypes

In [ ]:
cleanTypes(oil_df)
oil_df.dtypes

In [ ]:
# Get plastics data
plastics_nyse_tic = ['ATR','BERY','AWI','MYE','TUP']
plastics_df = getBasket(stock_dataset, plastics_nyse_tic,4)
plastics_df.head()

In [ ]:
plastics_df.isnull().sum()

In [ ]:
plastics_df.duplicated().sum()

In [ ]:
plastics_df.dtypes

In [ ]:
cleanTypes(plastics_df)
plastics_df.dtypes

In [ ]:
# Get Power Generator data
powergen_nyse_tic = ['LNT','EXC','NWE','XEL','D','EIX','ETR']
power_df = getBasket(stock_dataset, powergen_nyse_tic,4)
power_df

In [ ]:
power_df.isnull().sum()

In [ ]:
power_df.duplicated().sum()

In [ ]:
power_df.dtypes

In [ ]:
cleanTypes(power_df)
power_df.dtypes

In [ ]:
# Get Airline data
airline_nyse_tic = ['ALK','LUV','DAL','AAL','JBLU','UAL','SKYW','CPA']
air_df = getBasket(stock_dataset, airline_nyse_tic,4)
air_df.head()

In [ ]:
air_df.isnull().sum()

In [ ]:
air_df.duplicated().sum()

In [ ]:
air_df.dtypes

In [ ]:
cleanTypes(air_df)
air_df.dtypes

In [ ]:





airline_otc_tic = ['DLAKY','CPCAY','LTMAQ','AFLYY']